In [2]:
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime,timedelta
import arrow

In [1]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as pltz
import xarray as xar
import rioxarray

In [3]:
# 尝试读取 max 的nc文件
file_path=r'E:\02data\03typhoon\TY2022_2021010416\COVERAGE\fieldSurge_TY2022_2021010416_c0_p00_201809150900.nc'
# 21-05-04 mac的配置
# file_path=r'/Users/liusihan/data/typhoon_data/TY2022_2021010416/COVERAGE/fieldSurge_TY1822_2020042710_c0_p00.nc'

- 21-05-04 读取诸时风暴增水场出现错误:  
ERROR:  
ValueError: unable to decode time units 'days since 0001-1-0' with 'the default calendar'. Try opening your dataset with decode_times=False or installing cftime if it is not installed.

In [4]:
ds_xr=xar.open_dataset(file_path,decode_times=False)
ds_xr

<xarray.Dataset>
Dimensions:    (lat: 660, lon: 1080, times: 48)
Coordinates:
  * times      (times) float64 1.0 2.0 3.0 4.0 5.0 ... 44.0 45.0 46.0 47.0 48.0
Dimensions without coordinates: lat, lon
Data variables:
    latitude   (lat) float32 ...
    longitude  (lon) float32 ...
    surge      (times, lat, lon) float32 ...
Attributes:
    description:  Storm Surge Field (starting time: 2018-09-15 17:00:00)

+ 21-05-06  
目前存在以下几个问题:  
* 时间戳是 days，倒置数据精确性有偏差
* 手动创建一个 times 维度

In [5]:
start_dt=datetime(2020,9,15,9)
start_dt

datetime.datetime(2020, 9, 15, 9, 0)

In [6]:
np_dt_range=np.array([start_dt+timedelta(hours=i) for i in range(48)])
np_dt_range[:5]

array([datetime.datetime(2020, 9, 15, 9, 0),
       datetime.datetime(2020, 9, 15, 10, 0),
       datetime.datetime(2020, 9, 15, 11, 0),
       datetime.datetime(2020, 9, 15, 12, 0),
       datetime.datetime(2020, 9, 15, 13, 0)], dtype=object)

In [7]:
len(np_dt_range)

48

------

将新创建的 `date range` 赋值给 `dataset` -> `times` 维度

In [8]:
ds_xr.coords['times']=np_dt_range

In [9]:
ds_xr

<xarray.Dataset>
Dimensions:    (lat: 660, lon: 1080, times: 48)
Coordinates:
  * times      (times) datetime64[ns] 2020-09-15T09:00:00 ... 2020-09-17T08:0...
Dimensions without coordinates: lat, lon
Data variables:
    latitude   (lat) float32 15.008333 15.025 15.041667 ... 25.975 25.991667
    longitude  (lon) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
    surge      (times, lat, lon) float32 ...
Attributes:
    description:  Storm Surge Field (starting time: 2018-09-15 17:00:00)

----

In [10]:
ds_xr.to_netcdf(r'E:\02data\03typhoon\TY2022_2021010416\COVERAGE\fieldSurge_TY2022_2021010416_c0_p00_201809150900_converted.nc')

------

In [11]:
ds_xr=ds_xr.rename_vars({'latitude':'y','longitude':'x'})
ds_xr

<xarray.Dataset>
Dimensions:  (lat: 660, lon: 1080, times: 48)
Coordinates:
  * times    (times) datetime64[ns] 2020-09-15T09:00:00 ... 2020-09-17T08:00:00
Dimensions without coordinates: lat, lon
Data variables:
    y        (lat) float32 15.008333 15.025 15.041667 ... 25.975 25.991667
    x        (lon) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
    surge    (times, lat, lon) float32 ...
Attributes:
    description:  Storm Surge Field (starting time: 2018-09-15 17:00:00)

In [12]:
ds_xr=ds_xr.swap_dims({'lat':'y','lon':'x'})
ds_xr

<xarray.Dataset>
Dimensions:  (times: 48, x: 1080, y: 660)
Coordinates:
  * y        (y) float32 15.008333 15.025 15.041667 ... 25.975 25.991667
  * x        (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
  * times    (times) datetime64[ns] 2020-09-15T09:00:00 ... 2020-09-17T08:00:00
Data variables:
    surge    (times, y, x) float32 ...
Attributes:
    description:  Storm Surge Field (starting time: 2018-09-15 17:00:00)

In [13]:
ds_xr.rio.set_spatial_dims("x","y",inplace=True)

<xarray.Dataset>
Dimensions:  (times: 48, x: 1080, y: 660)
Coordinates:
  * y        (y) float32 15.008333 15.025 15.041667 ... 25.975 25.991667
  * x        (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
  * times    (times) datetime64[ns] 2020-09-15T09:00:00 ... 2020-09-17T08:00:00
Data variables:
    surge    (times, y, x) float32 ...
Attributes:
    description:  Storm Surge Field (starting time: 2018-09-15 17:00:00)

In [14]:
ds_xr=ds_xr.rio.write_crs("epsg:4326", inplace=True)
ds_xr

<xarray.Dataset>
Dimensions:      (times: 48, x: 1080, y: 660)
Coordinates:
  * y            (y) float32 15.008333 15.025 15.041667 ... 25.975 25.991667
  * x            (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
  * times        (times) datetime64[ns] 2020-09-15T09:00:00 ... 2020-09-17T08...
    spatial_ref  int32 0
Data variables:
    surge        (times, y, x) float32 ...
Attributes:
    description:   Storm Surge Field (starting time: 2018-09-15 17:00:00)
    grid_mapping:  spatial_ref

* 注意此处缺少 coords,设置了坐标系

In [15]:
print(ds_xr.info())

xarray.Dataset {
dimensions:
	times = 48 ;
	x = 1080 ;
	y = 660 ;

variables:
	float32 y(y) ;
		y:units = N ;
	float32 x(x) ;
		x:units = E ;
	datetime64[ns] times(times) ;
	float32 surge(times, y, x) ;
		surge:units = m ;
		surge:grid_mapping = spatial_ref ;
	int32 spatial_ref() ;
		spatial_ref:crs_wkt = GEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]],AXIS["geodetic longitude (Lon)",east,ORDER[2],ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]] ;
		spatial_ref:semi_major_axis = 6378137.0 ;
		spatial_ref:semi_minor_axis = 6356752.314245179 ;
		spatial_ref:inverse_flattening = 298.257223563 ;
		spatial_ref:reference_ellipsoid_name = WGS 84 ;
		spatial_ref:longitude_of_prime_meridian = 0.0 ;
		spatial_ref:prime_meridian_name = Greenwich ;
		spatial_r

注意使用 rioxarray -> geotiff 时，需要将 lon -> x, lat -> y

注意此处还需要加入一个步骤，对 y 也就是经度倒叙排列

In [16]:
ds_xr['y']=ds_xr['y'][::-1]

In [17]:
ds_xr

<xarray.Dataset>
Dimensions:      (times: 48, x: 1080, y: 660)
Coordinates:
  * y            (y) float32 25.991667 25.975 25.958334 ... 15.025 15.008333
  * x            (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
  * times        (times) datetime64[ns] 2020-09-15T09:00:00 ... 2020-09-17T08...
    spatial_ref  int32 0
Data variables:
    surge        (times, y, x) float32 ...
Attributes:
    description:   Storm Surge Field (starting time: 2018-09-15 17:00:00)
    grid_mapping:  spatial_ref

In [17]:
ds_xr['times']

<xarray.DataArray 'times' (times: 48)>
array(['2018-09-15T09:00:00.000000000', '2018-09-15T10:00:00.000000000',
       '2018-09-15T11:00:00.000000000', '2018-09-15T12:00:00.000000000',
       '2018-09-15T13:00:00.000000000', '2018-09-15T14:00:00.000000000',
       '2018-09-15T15:00:00.000000000', '2018-09-15T16:00:00.000000000',
       '2018-09-15T17:00:00.000000000', '2018-09-15T18:00:00.000000000',
       '2018-09-15T19:00:00.000000000', '2018-09-15T20:00:00.000000000',
       '2018-09-15T21:00:00.000000000', '2018-09-15T22:00:00.000000000',
       '2018-09-15T23:00:00.000000000', '2018-09-16T00:00:00.000000000',
       '2018-09-16T01:00:00.000000000', '2018-09-16T02:00:00.000000000',
       '2018-09-16T03:00:00.000000000', '2018-09-16T04:00:00.000000000',
       '2018-09-16T05:00:00.000000000', '2018-09-16T06:00:00.000000000',
       '2018-09-16T07:00:00.000000000', '2018-09-16T08:00:00.000000000',
       '2018-09-16T09:00:00.000000000', '2018-09-16T10:00:00.000000000',
       '2018-09-16T11:00:00.000000000', '2018-09-16T12:00:00.000000000',
       '2018-09-16T13:00:00.000000000', '2018-09-16T14:00:00.000000000',
       '2018-09-16T15:00:00.000000000', '2018-09-16T16:00:00.000000000',
       '2018-09-16T17:00:00.000000000', '2018-09-16T18:00:00.000000000',
       '2018-09-16T19:00:00.000000000', '2018-09-16T20:00:00.000000000',
       '2018-09-16T21:00:00.000000000', '2018-09-16T22:00:00.000000000',
       '2018-09-16T23:00:00.000000000', '2018-09-17T00:00:00.000000000',
       '2018-09-17T01:00:00.000000000', '2018-09-17T02:00:00.000000000',
       '2018-09-17T03:00:00.000000000', '2018-09-17T04:00:00.000000000',
       '2018-09-17T05:00:00.000000000', '2018-09-17T06:00:00.000000000',
       '2018-09-17T07:00:00.000000000', '2018-09-17T08:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * times        (times) datetime64[ns] 2018-09-15T09:00:00 ... 2018-09-17T08...
    spatial_ref  int32 0

In [39]:
ds_xr['times'][0]

<xarray.DataArray 'times' ()>
array('2018-09-15T18:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    times        datetime64[ns] 2018-09-15T18:00:00
    spatial_ref  int32 0

In [18]:
dt_first_test=pd.to_datetime(ds_xr.coords['times'][0].values)
dt_first_test

Timestamp('2018-09-15 09:00:00')

In [19]:
arrow.get(dt_first_test).format('YYYYMMDDHHmm')

'201809150900'

In [65]:
pd.to_datetime(ds_xr.coords['times'][0].values)

Timestamp('2018-09-15 18:00:00')

In [66]:
ds_xr.sel(times=pd.to_datetime(ds_xr.coords['times'][0].values))

<xarray.Dataset>
Dimensions:      (x: 1080, y: 660)
Coordinates:
  * y            (y) float32 25.991667 25.975 25.958334 ... 15.025 15.008333
  * x            (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
    times        datetime64[ns] 2018-09-15T18:00:00
    spatial_ref  int32 0
Data variables:
    surge        (y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.04 0.04 0.04 0.04 0.04
Attributes:
    description:   Storm Surge Field
    grid_mapping:  spatial_ref

In [19]:
from pathlib import Path

In [72]:
file_name=f'fieldSurge_TY1822_2020042710_c0_p00_converted_{123}.tif'
file_name

'fieldSurge_TY1822_2020042710_c0_p00_converted_123.tif'

In [79]:
p=Path(r'E:\02data\03typhoon\TY2022_2021010416\COVERAGE')

In [80]:
p/file_name

WindowsPath('E:/02data/03typhoon/TY2022_2021010416/COVERAGE/fieldSurge_TY1822_2020042710_c0_p00_converted_123.tif')

In [20]:
# 遍历生成 tif 
for temp in ds_xr['times']:
    dt_temp=pd.to_datetime(temp.values)
    dt_str=arrow.get(dt_temp).format('YYYYMMDDHHmm')
    print(dt_str)
    ds_xr_temp=ds_xr.sel(times=pd.to_datetime(temp.values))    
    p=Path(r'E:\02data\03typhoon\TY2022_2021010416\COVERAGE')
    file_name=f'fieldSurge_TY2022_2021010416_c0_p00_201809150900_converted_{dt_str}.tif'
    full_path=str(p/file_name)
    print(f'最后输出的目录为{full_path}')
    ds_xr_temp.rio.to_raster(full_path)
    print('-------------')
# ds_xr.rio.to_raster(r'/Users/liusihan/data/typhoon_data/TY2022_2021010416/COVERAGE/maxSurge_TY2022_2021010416_c0_p00.tif')

202009150900
最后输出的目录为E:\02data\03typhoon\TY2022_2021010416\COVERAGE\fieldSurge_TY2022_2021010416_c0_p00_201809150900_converted_202009150900.tif
-------------
202009151000
最后输出的目录为E:\02data\03typhoon\TY2022_2021010416\COVERAGE\fieldSurge_TY2022_2021010416_c0_p00_201809150900_converted_202009151000.tif
-------------
202009151100
最后输出的目录为E:\02data\03typhoon\TY2022_2021010416\COVERAGE\fieldSurge_TY2022_2021010416_c0_p00_201809150900_converted_202009151100.tif
-------------
202009151200
最后输出的目录为E:\02data\03typhoon\TY2022_2021010416\COVERAGE\fieldSurge_TY2022_2021010416_c0_p00_201809150900_converted_202009151200.tif
-------------
202009151300
最后输出的目录为E:\02data\03typhoon\TY2022_2021010416\COVERAGE\fieldSurge_TY2022_2021010416_c0_p00_201809150900_converted_202009151300.tif
-------------
202009151400
最后输出的目录为E:\02data\03typhoon\TY2022_2021010416\COVERAGE\fieldSurge_TY2022_2021010416_c0_p00_201809150900_converted_202009151400.tif
-------------
202009151500
最后输出的目录为E:\02data\03typhoon\TY2022_2021

In [13]:
rio_df=rioxarray.open_rasterio(file_path)
rio_df

<xarray.DataArray 'pro_surge' (band: 1, y: 660, x: 1080)>
[712800 values with dtype=float32]
Coordinates:
  * band         (band) int32 1
  * y            (y) float64 0.5 1.5 2.5 3.5 4.5 ... 656.5 657.5 658.5 659.5
  * x            (x) float64 0.5 1.5 2.5 3.5 ... 1.078e+03 1.078e+03 1.08e+03
    spatial_ref  int32 0
Attributes:
    units:         %
    _FillValue:    999.0
    scale_factor:  1.0
    add_offset:    0.0
    grid_mapping:  spatial_ref

In [ ]:
rio_df.to_